In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

### 读取原始数据

In [2]:
df = pd.read_csv("Tianchi_power.csv")

df9 = pd.read_csv("Tianchi_power_9.csv")

df = pd.concat([df,df9])

df['record_date'] = pd.to_datetime(df['record_date'])
df.head()


,record_date,user_id,power_consumption
0,2015-01-01,1,1135
1,2015-01-02,1,570
2,2015-01-03,1,3418
3,2015-01-04,1,3968
4,2015-01-05,1,3986


In [3]:
# feature = pd.read_csv("feature_isworkday.csv")
# feature.shape

### 填充缺省值

In [4]:
dates = pd.date_range("2015-01-01","2016-09-30", freq="1D")
userid = pd.DataFrame({'user_id':df['user_id'].unique()})
dates = pd.DataFrame(dates)
dates.columns = ['record_date']
userid['key'] = 0
dates['key'] = 0
full = userid.merge(dates, how='left', on = 'key')
full.drop('key',1, inplace=True)
full.head(2)

,user_id,record_date
0,1,2015-01-01
1,1,2015-01-02


In [5]:
df = pd.merge(full, df, how='left', on=['user_id','record_date'])
df['power_consumption'] = df['power_consumption'].interpolate()
df.shape

(929106, 3)

### 填充特征

In [6]:
df['dow'] = df['record_date'].apply(lambda x: x.dayofweek)
df['doy'] = df['record_date'].apply(lambda x: x.dayofyear)
df['day'] = df['record_date'].apply(lambda x: x.day)
df['month'] = df['record_date'].apply(lambda x: x.month)
df['year'] = df['record_date'].apply(lambda x: x.year)

def map_season(month):
    month_dic = {1:1, 2:1, 3:2, 4:2, 5:3, 6:3, 7:3, 8:3, 9:3, 10:4, 11:4, 12:1}
    return month_dic[month]

df['season'] = df['month'].apply(lambda x: map_season(x))

In [7]:
### 用电量聚合到一个月

base_df = df[['record_date','power_consumption']].groupby(by='record_date').agg('sum')
base_df = base_df.reset_index()
base_df.head()
base_df.shape
# df = base_df

(639, 2)

### 拼接测试集

In [8]:
df_test = base_df[(base_df.record_date>='2016-08-31')&(base_df.record_date<='2016-09-30')]
df_test['record_date'] = pd.DataFrame(df_test['record_date']+pd.Timedelta('31 days'))
df_test#.shape#head()


,record_date,power_consumption
608,2016-10-01,3664003.0
609,2016-10-02,3683282.0
610,2016-10-03,3723110.0
611,2016-10-04,3490429.0
612,2016-10-05,3258102.0
613,2016-10-06,3681162.0
614,2016-10-07,3582158.0
615,2016-10-08,3480405.0
616,2016-10-09,3584115.0
617,2016-10-10,3729301.0


In [9]:
df = pd.concat([base_df, df_test]).sort_values(['record_date'])
# base_df.shape

# df = base_df
df.head()


,record_date,power_consumption
0,2015-01-01,2900575.0
1,2015-01-02,3158211.0
2,2015-01-03,3596487.0
3,2015-01-04,3939672.0
4,2015-01-05,4101790.0


In [10]:
base_df = df
df.head()

,record_date,power_consumption
0,2015-01-01,2900575.0
1,2015-01-02,3158211.0
2,2015-01-03,3596487.0
3,2015-01-04,3939672.0
4,2015-01-05,4101790.0


### 造特征

In [11]:
base_df['dow'] = base_df['record_date'].apply(lambda x: x.dayofweek)
base_df['doy'] = base_df['record_date'].apply(lambda x: x.dayofyear)
base_df['day'] = base_df['record_date'].apply(lambda x: x.day)
base_df['month'] = base_df['record_date'].apply(lambda x: x.month)
base_df['year'] = base_df['record_date'].apply(lambda x: x.year)

def map_season(month):
    month_dic = {1:1, 2:1, 3:2, 4:2, 5:3, 6:3, 7:3, 8:3, 9:3, 10:4, 11:4, 12:1}
    return month_dic[month]

base_df['season'] = base_df['month'].apply(lambda x: map_season(x))

### 造特征  添加特征 是否为上班日 

In [12]:
# # 添加特征 是否为上班日 

# base_df['isworkday'] = feature['isworkday']
# base_df[base_df.isworkday==0].index
# base_df#.shape#()

In [13]:
# base_df['holiday_usage'] = base_df['power_consumption']
# base_df.loc[base_df[base_df.isworkday==1].index ,'holiday_usage'] = 0

# base_df.shape

In [14]:
# base_df[(base_df.record_date>='2015-10-01') & (base_df.record_date<='2015-10-07')]['holiday_usage'].values

In [15]:
# base_df.loc[base_df[(base_df.record_date>='2016-10-01') & (base_df.record_date<='2016-10-07')].index ,'holiday_usage'] = base_df[(base_df.record_date>='2015-10-01') & (base_df.record_date<='2015-10-07')]['holiday_usage']

In [16]:
# plt.plot(base_df['holiday_usage'],'-o')
# base_df[(base_df.record_date>='2016-10-01') & (base_df.record_date<='2016-10-07')].index
# base_df.loc[base_df[(base_df.record_date>='2016-10-01') & (base_df.record_date<='2016-10-07')].index ,'holiday_usage'] 

### 添加更多特征
### 先添加前一个月的均值和方差

In [17]:
df_stats = base_df[['power_consumption','year','month']].groupby(by=['year', 'month']).agg(['mean', 'std'])
df_stats.head()

df_stats.columns = df_stats.columns.droplevel(0)
df_stats = df_stats.reset_index()
df_stats.head()
df_stats['mean'] = df_stats['mean'].shift(1)
# df_stats['2_m_mean'] = df_stats['mean'].shift(2)
df_stats['std'] = df_stats['std'].shift(1)
# df_stats['2_m_std'] = df_stats['std'].shift(2)
df_stats.head()


,year,month,mean,std
0,2015,1,NaN,NaN
1,2015,2,3.961383e+06,303629.486622
2,2015,3,2.795163e+06,769697.864999
3,2015,4,3.478852e+06,564753.747993
4,2015,5,3.768333e+06,190787.832757


In [18]:
# # train = pd.merge('base_df','df_stats',on=['year','month'])
# train = pd.merge(base_df, df_stats, how='left', on=['year','month'])
# train.head()

### 把一个月分成上中下旬3个month_periods 或者 4周总共4个week_periods
### 用它们的均值之差表示数据上下起伏趋势

In [19]:
# 先对一个月做切分
def month_period(day):
    if day>=1 and day<=10:
        return 1
    elif day>=11 and day<=20:
        return 2
    else:
        return 3

def week_period(day):
    if day>=1 and day<=7:
        return 1
    elif day>=8 and day<=14:
        return 2
    elif day>=15 and day<=21:
        return 3
    else:
        return 4

base_df['month_period'] = base_df['day'].apply(lambda x:month_period(x))
base_df['week_period'] = base_df['day'].apply(lambda x:week_period(x))

### 按照period求均值

In [20]:
month_period_df = base_df[['power_consumption','year','month','month_period']].groupby(by=['year', 'month', 'month_period']).agg('mean')
month_period_df = month_period_df.reset_index()
month_period_df.head()

,year,month,month_period,power_consumption
0,2015,1,1,3.825766e+06
1,2015,1,2,4.016926e+06
2,2015,1,3,4.034177e+06
3,2015,2,1,3.691394e+06
4,2015,2,2,2.338909e+06


### 求出来均值之差

In [21]:
month_period_df['month_period'] = month_period_df['power_consumption'].diff(1)
month_period_df = month_period_df.drop('power_consumption',1)
month_period_df.head()

,year,month,month_period
0,2015,1,NaN
1,2015,1,1.911600e+05
2,2015,1,1.725125e+04
3,2015,2,-3.427832e+05
4,2015,2,-1.352486e+06


### 填充上个月和上上个月的趋势数据

In [22]:
month_period_df = month_period_df[['year','month','month_period']].groupby(by=['year','month']).agg(lambda x: ",".join(x.apply(str).values.tolist()))
month_period_df = month_period_df.reset_index()
month_period_df[['month_period_1','month_period_2','month_period_3']] =  month_period_df['month_period'].str.split(',', expand=True) 
month_period_df = month_period_df.drop('month_period',1)

# month_1_columns = [tmp+'_1month' for tmp in ['month_period_1', 'month_period_2','month_period_3']]
month_period_df[['month_period_1', 'month_period_2','month_period_3']] = month_period_df[['month_period_1', 'month_period_2','month_period_3']].shift(1)
# month_2_columns = [tmp+'_2month' for tmp in ['month_period_1', 'month_period_2','month_period_3']]
# month_period_df[month_2_columns] = month_period_df[['month_period_1', 'month_period_2','month_period_3']].shift(2)

month_period_df.head()

,year,month,month_period_1,month_period_2,month_period_3
0,2015,1,NaN,NaN,NaN
1,2015,2,nan,191160.0,17251.2545454544
2,2015,3,-342783.1545454548,-1352485.6999999997,-93716.5375000001
3,2015,4,863222.6375000002,559158.0499999998,-23522.659090909176
4,2015,5,109222.00909090927,-25469.75,96123.1000000001


In [23]:
# # 合并
# train_mon =  pd.merge(train, month_period_df, how='left', on=['year','month'])
# train_mon.head()


In [24]:
week_period_df = base_df[['power_consumption','year','month','week_period']].groupby(by=['year', 'month', 'week_period']).agg('mean')
week_period_df = week_period_df.reset_index()
# week_period_df.head()

In [25]:
week_period_df['week_period'] = week_period_df['power_consumption'].diff(1)
week_period_df = week_period_df.drop('power_consumption',1)
# week_period_df.head()

In [26]:
# 将每个月中四个周的值分为四列。
week_period_df = week_period_df[['year','month','week_period']].groupby(by=['year','month']).agg(lambda x: ",".join(x.apply(str).values.tolist()))
week_period_df = week_period_df.reset_index()
week_period_df[['week_period_1','week_period_2','week_period_3','week_period_4']] =  week_period_df['week_period'].str.split(',', expand=True) 
week_period_df = week_period_df.drop('week_period',1)

add_columns_1 = [tmp+'_1month' for tmp in ['week_period_1', 'week_period_2','week_period_3', 'week_period_4']]
week_period_df[['week_period_1','week_period_2','week_period_3','week_period_4']] = week_period_df[['week_period_1', 'week_period_2','week_period_3', 'week_period_4']].shift(1)
# add_columns_2 = [tmp+'_2month' for tmp in ['week_period_1', 'week_period_2','week_period_3', 'week_period_4']]
# week_period_df[add_columns_2] = week_period_df[['week_period_1', 'week_period_2','week_period_3', 'week_period_4']].shift(2)

week_period_df.head()



,year,month,week_period_1,week_period_2,week_period_3,week_period_4
0,2015,1,NaN,NaN,NaN,NaN
1,2015,2,nan,297703.42857142864,46736.28571428545,-31404.27142857155
2,2015,3,-220744.0142857139,-739169.8571428573,-1042483.8571428573,252348.35714285728
3,2015,4,645923.0714285714,821015.2142857141,-292440.07142857136,245783.85714285728
4,2015,5,-9620.857142857276,113845.42857142864,-66743.64285714272,91990.90476190485


In [27]:
# 添加 week period 相关特征
# df_final = pd.merge(train_mon, week_period_df, how='left', on=['year','month']) 

In [28]:
df_final = base_df
df_final.head()

,record_date,power_consumption,dow,doy,day,month,year,season,month_period,week_period
0,2015-01-01,2900575.0,3,1,1,1,2015,1,1,1
1,2015-01-02,3158211.0,4,2,2,1,2015,1,1,1
2,2015-01-03,3596487.0,5,3,3,1,2015,1,1,1
3,2015-01-04,3939672.0,6,4,4,1,2015,1,1,1
4,2015-01-05,4101790.0,0,5,5,1,2015,1,1,1


In [29]:
df_final.columns

Index(['record_date', 'power_consumption', 'dow', 'doy', 'day', 'month',
       'year', 'season', 'month_period', 'week_period'],
      dtype='object')

### 去掉不需要的数据

In [30]:
df_final = df_final[df_final.record_date>='2015-03-01']   #######################################
# df_final['previous_2months_diff'] = df_final['1_m_mean']-df_final['2_m_mean']
df_final.head()

,record_date,power_consumption,dow,doy,day,month,year,season,month_period,week_period
59,2015-03-01,2900464.0,6,60,1,3,2015,2,1,1
60,2015-03-02,3334082.0,0,61,2,3,2015,2,1,1
61,2015-03-03,3492606.0,1,62,3,3,2015,2,1,1
62,2015-03-04,3597890.0,2,63,4,3,2015,2,1,1
63,2015-03-05,1798044.0,3,64,5,3,2015,2,1,1


In [31]:
df_final = df_final.drop(['record_date'],1)
df_final.to_csv('train.csv', index=False)

In [32]:
#### RF模型

In [36]:
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
# from sklearn.model_selection import cr
from datetime import datetime 

# 输入原始数据集 包含 train set 集， 并有label列
def get_train_set(df_final):
        
    final_train_data = df_final[~((df_final.year==2016)&(df_final.month==10))].drop(['power_consumption'],1)
    final_train_data = final_train_data.astype(int)
    final_test_data = df_final[((df_final.year==2016)&(df_final.month==10))].drop(['power_consumption'],1)
    final_test_data = final_test_data.astype(int)
    train_target = df_final[~((df_final.year==2016)&(df_final.month==10))][['power_consumption']]
    
    # # strcolumns = ['dow','doy','day','month','year','isworkday','season']
    # strcolumns = ['dow','doy','day','month','year','season','isholiday']
    
    # train = final_train_data.copy()
    # train[strcolumns] = train[strcolumns].astype(str)
    # test = final_test_data.copy()
    # test[strcolumns] = test[strcolumns].astype(str)
    
    # train_target
    
    train_X, train_y = final_train_data, train_target
    test_X = final_test_data
    return train_X, train_y ,test_X


def get_predict(model, train_X, train_y, test_X):
    # test_X
    # {'n_estimators': 200, 'max_depth': 4}
    # {'max_features': 'log2', 'n_estimators': 600, 'max_depth': 12}
    
    # reg = RandomForestRegressor(n_estimators=600, max_features='log2', max_depth=12 )
    # reg = RandomForestRegressor(n_estimators=200)
#     reg = model
    model.fit(train_X,train_y)
    
    # y_predict = reg.best_estimator_.predict(test_X)
    y_predict = model.predict(test_X) # bagging_reg
    return y_predict




def timetransform(t):
        t = str(t)[0:10]
        
        try:
            time = datetime.strptime(t, '%Y-%m-%d')
            t = time.strftime('%Y%m%d')
        except:
            pass
        return t

def save_predict(y_predict, filename):

        # step 3
    commit_df = pd.date_range('2016/10/1', periods=31, freq='D')
    commit_df = pd.DataFrame(commit_df)
    commit_df.columns = ['predict_date']
    commit_df['predict_power_consumption'] = pd.DataFrame(y_predict).astype('int')
    # y_predict

    curtime = datetime.today()
    strtime = curtime.strftime('%m%d')
    
    
    commit_df['predict_date'] = commit_df['predict_date'].apply(timetransform)
    
    #     commit_df.head()
    # predict_et_pcdata_0611
    # commit_df.to_csv('./predict/predict_' + strtime + '.csv',index=False) # to 记录下每一次结果 
    commit_df.to_csv(filename, index=False) # 同时生成待提交的结果文件，每次新结果覆盖老文件
    

In [43]:

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score, train_test_split
import matplotlib.pyplot as plt

# {'max_features': 0.8, 'n_estimators': 800}
mods = [ExtraTreesRegressor(n_estimators=400), 
        RandomForestRegressor(n_estimators=400),
       ]
legend = []

plt.figure(figsize=(15,8))
x = range(31)
y = []
with open('10.txt','r') as f:
    lines = f.readlines()
    for line in lines:
        y.append(int(''.join(line.split('\t')[-1].split())))
plt.plot(x, y,'-g<')
legend.append('10')

def plot_predict(ds, mod, flag=''):
    curtime = datetime.today()
    strtime = curtime.strftime('%m%d_%H%M')
    fs = 'predict_rf_'+strtime+'.csv'
    
#     for ds in dfs:
    train_X, train_y ,test_X = get_train_set(ds)
#     for mod, filename in zip(mods, fs):
    predict = get_predict(mod, train_X, train_y ,test_X)
    save_predict(predict, fs)
#             x = prediction['predict_date']#.values
#     x = test_X['predict_power_consumption']#.values
    x = range(31)
    print(predict)
    plt.plot(x, predict)
    legend.append(flag)
#     plt.show()

for i in range(5,7):
    tmpd = '2015-0'+ str(i) +'-01'
    ds = df[df.record_date>=tmpd] 
    ds = ds.drop(['record_date'],1)
    plot_predict(ds, mods[1], tmpd)


plt.legend(legend)
plt.show()

# 通过这个图查看去掉不同月份的数据后预测的曲线图的变化情况， 看上去总体趋势没有太大变化， 只是上下平移而已

[ 3166491.515    3142594.845    3401925.88     3566815.13     3649771.3475
  3707540.8025   3760136.915    3734423.2125   3452642.30875  3696826.5525
  3708908.66     3723511.065    3700115.1225   3673036.1025   3591245.0575
  3530758.9175   3764485.8725   3788956.5425   3788266.805    3868456.075
  3911002.06     3863223.78     3566776.445    3911219.23375  3922060.50875
  3923813.25625  3917750.7      3881544.96     3756860.34     3542220.19875
  3750128.14625]
[ 2971108.175    2999579.4575   3254606.195    3486908.7975   3595816.5125
  3685012.4925   3741988.6925   3717275.905    3415394.2425   3646960.0525
  3668198.5225   3682287.3225   3654987.3025   3640109.2525   3557969.7275
  3473891.70875  3698369.7      3738933.9425   3746341.2475   3849194.83
  3905913.795    3858955.825    3531194.17     3876675.7575   3894938.4525
  3904570.84     3921773.2125   3875935.805    3710638.93625  3491913.29375
  3708245.94375]


### 特征重要度排序

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
print('Plot feature importances...')
ax = lgb.plot_importance(gbm, max_num_features=10)
plt.show()


In [ ]:
commit_df = pd.date_range('2016/10/1', periods=31, freq='D')
commit_df = pd.DataFrame(commit_df)
commit_df.columns = ['predict_date']
y_predict = gbm.predict(test_lgb.values)
commit_df['predict_power_consumption'] = pd.DataFrame(y_predict).astype('int')
commit_df

In [ ]:
# 生成提交结果文件
# 转换日期格式 

from datetime import datetime 

def timetransform(t):
    t = str(t)[0:10]
    time = datetime.strptime(t, '%Y-%m-%d')
    res = time.strftime('%Y%m%d')
    return res

commit_df['predict_date'] = commit_df['predict_date'].apply(timetransform)

commit_df.head()
commit_df.to_csv('Tianchi_power_predict_table.csv',index=False)

In [ ]:
# 读入最好的成绩 做参考
# ref = pd.read_csv('predict_liuxin.csv')
# test_x = pd.read_csv('test.csv')
# train_y = train['power_consumption']
# train_X = train.drop('power_consumption',axis=1)
# train_X.shape
# ref

In [ ]:
# 读取当前预测结果
train = pd.read_csv('Tianchi_power_predict_table.csv')
# test_x = pd.read_csv('test.csv')
# train

In [ ]:
y2 = train['predict_power_consumption'].values

x = train['predict_date'].values

# y = ref['predict_power_consumption'].values

# plt.plot(x, y)
plt.plot(x,y2)
plt.legend()

